In [1]:
import s4sim.hardware.config
c=s4sim.hardware.config.get_example()

In [2]:
import s4sim

In [3]:
s4sim.__version__

'0+untagged.154.gca07638'

Using commit:

```
commit ca07638f6781144648938a636a972711321ba166 (HEAD -> 85-tube_LAT, 
origin/85-tube_LAT)
Author: smsimon <smsimon@umich.edu>
Date:   Mon Feb 8 09:41:21 2021 -0600

    Effective flat band bandpasses and updated noise levels for SPSAT and SPLAT. Calculated at elevation of 50 deg 
```

In [4]:
c.data["telescopes"]["SAT0"]["fwhm"]

OrderedDict([('LFS1', 72.8),
             ('LFS2', 72.8),
             ('MFLS1', 25.5),
             ('MFLS2', 25.5),
             ('MFHS1', 22.7),
             ('MFHS2', 22.7),
             ('HFS1', 13.0),
             ('HFS2', 13.0)])

In [5]:
c.data["telescopes"]["SAT3"]["fwhm"].keys()

odict_keys(['LFS1', 'LFS2', 'MFLS1', 'MFLS2', 'MFHS1', 'MFHS2', 'HFS1', 'HFS2'])

In [6]:
c.data["telescopes"]["SAT3"].keys()

odict_keys(['tubes', 'platescale', 'tubespace', 'fwhm'])

In [7]:
c.data["bands"].keys()

odict_keys(['ULFPL1', 'LFL1', 'LFL2', 'LFPL1', 'LFPL2', 'LFS1', 'LFS2', 'MFL1', 'MFL2', 'MFPL1', 'MFPL2', 'MFLS1', 'MFLS2', 'MFHS1', 'MFHS2', 'HFL1', 'HFL2', 'HFPL1', 'HFPL2', 'HFS1', 'HFS2'])

In [8]:
c.data["bands"]["LFL1"]

OrderedDict([('center', 25.75),
             ('low', 21.5),
             ('high', 30.0),
             ('bandpass', ''),
             ('NET', 287.4),
             ('fknee', 50.0),
             ('fmin', 0.01),
             ('alpha', 3.5),
             ('A', 0.09),
             ('C', 0.87)])

In [9]:
import numpy as np

# NET and number of detectors per tube

Copy-pasted from https://docs.google.com/spreadsheets/d/1X0x8wYhjHdI_WPjhgbtyVlVQrX86tTgxgqEUhan2Hxg/edit?usp=sharing
On 10 February 2021

In [10]:
NUM_DET = {}
SAT_NUM_DET_STRING = "288	288	3528	3528	3528	3528	8592	8592"
for sat_band, num_det in zip(c.data["telescopes"]["SAT3"]["fwhm"].keys(), SAT_NUM_DET_STRING.split()):
    NUM_DET[sat_band] = int(num_det)

For LAT copy-pasted from line 600-610 of `hardware/config.py` of `s4sim` at the same hash used above, see https://github.com/CMB-S4/s4sim/issues/15#issuecomment-776901802

In [11]:
wnp = {
    "ULFPL": 27,                                                                                                                                         
    "LFL": 48,
    "LFPL": 48,
    "LFS": 12,
    "MFL": 432,
    "MFPL": 432,
    "MFLS": 147,
    "MFHS": 147,
    "HFL": 432,
    "HFPL": 432,
    "HFS": 469,
}

In [12]:
for band in c.data["telescopes"]["LAT0"]["fwhm"].keys():
    NUM_DET[band] = wnp[band[:-1]]*2

In [13]:
c.data["telescopes"]["LAT0"]["fwhm"].keys()

odict_keys(['ULFPL1', 'LFL1', 'LFPL1', 'LFL2', 'LFPL2', 'MFL1', 'MFPL1', 'MFL2', 'MFPL2', 'HFL1', 'HFPL1', 'HFL2', 'HFPL2'])

In [14]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [15]:
ipac_warning = ["Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')"]

In [16]:
c.data["telescopes"]["LAT0"]["fwhm"]

OrderedDict([('ULFPL1', 10.0),
             ('LFL1', 7.4),
             ('LFPL1', 7.4),
             ('LFL2', 5.1),
             ('LFPL2', 5.1),
             ('MFL1', 2.2),
             ('MFPL1', 2.2),
             ('MFL2', 1.4),
             ('MFPL2', 1.4),
             ('HFL1', 1.0),
             ('HFPL1', 1.0),
             ('HFL2', 0.9),
             ('HFPL2', 0.9)])

In [17]:
s4 = []
for i, (band, v) in enumerate(c.data["bands"].items()):
    print("## " + band)
    print("Bandpass [GHz] low,center,high: ", v["low"], v["center"], v["high"])
    try:
        tel = "SAT"
        fwhm = c.data["telescopes"][tel + "0"]["fwhm"][band]
    except:
        tel = "LAT"
        fwhm = c.data["telescopes"][tel + "0"]["fwhm"][band]
    print("Beam [arcmin]: ", fwhm)
    d = OrderedDict(band=band)
    d["band"] = band
    d["telescope"] = tel
    d["center_frequency"] = v["center"] * u.GHz
    d["fwhm"] = fwhm * u.arcmin
    d["detectors_per_tube"] = NUM_DET[band]
    d["NET"] = v["NET"] * u.uK*u.s**.5
    bandpass_table = QTable(dict(bandpass_frequency=np.linspace(v["low"], v["high"], 10) * u.GHz,
    bandpass_weight=np.ones(10, dtype=np.float)))
    bandpass_table.meta["comments"] = ipac_warning
    bandpass_table.write(f"../instrument_model/bandpass_{band}.tbl", format="ascii.ipac", overwrite=True)
    s4.append(d)

## ULFPL1
Bandpass [GHz] low,center,high:  17.5 20.0 22.5
Beam [arcmin]:  10.0
## LFL1
Bandpass [GHz] low,center,high:  21.5 25.75 30.0
Beam [arcmin]:  7.4
## LFL2
Bandpass [GHz] low,center,high:  30.0 38.75 47.5
Beam [arcmin]:  5.1
## LFPL1
Bandpass [GHz] low,center,high:  21.5 25.75 30.0
Beam [arcmin]:  7.4
## LFPL2
Bandpass [GHz] low,center,high:  30.0 38.75 47.5
Beam [arcmin]:  5.1
## LFS1
Bandpass [GHz] low,center,high:  21.5 25.75 30.0
Beam [arcmin]:  72.8
## LFS2
Bandpass [GHz] low,center,high:  30.0 38.75 47.5
Beam [arcmin]:  72.8
## MFL1
Bandpass [GHz] low,center,high:  77.0 91.5 106.0
Beam [arcmin]:  2.2
## MFL2
Bandpass [GHz] low,center,high:  128.0 148.5 169.0
Beam [arcmin]:  1.4
## MFPL1
Bandpass [GHz] low,center,high:  77.0 91.5 106.0
Beam [arcmin]:  2.2
## MFPL2
Bandpass [GHz] low,center,high:  128.0 148.5 169.0
Beam [arcmin]:  1.4
## MFLS1
Bandpass [GHz] low,center,high:  74.8 85.0 95.2
Beam [arcmin]:  25.5
## MFLS2
Bandpass [GHz] low,center,high:  129.1 145.0 161.0
Bea

In [18]:
cat ../instrument_model/bandpass_HFL1.tbl

\ Text file in IPAC table format, read with astropy
\ from astropy.table import QTable
\ QTable.read('filename.tbl', format='ascii.ipac')
|bandpass_frequency|bandpass_weight|
|            double|         double|
|               GHz|               |
|              null|           null|
              198.0             1.0 
 204.44444444444446             1.0 
 210.88888888888889             1.0 
 217.33333333333334             1.0 
 223.77777777777777             1.0 
 230.22222222222223             1.0 
 236.66666666666669             1.0 
 243.11111111111111             1.0 
 249.55555555555554             1.0 
              256.0             1.0 


# Sensitivity from Decadal Survey Report

For verification purposes we want to compare simulation results to the expected performance as detailed in the DSR.
The DSR is available in tex format in the private Github repository: https://github.com/CMB-S4/DecadalSurveyReport

Numbers extracted on 26 January 2021 from version `83d362e2301b80444ed09c29293998cf5917d899`

Changed the frequency labels for LAT

In [19]:
from io import StringIO
import pandas as pd

def parse_table(table_string):
    """Parse Latex table into a Pandas Dataframe"""
    table = pd.read_csv(StringIO(table_string), sep="&", index_col=0, skipinitialspace=True).T
    table.index = pd.to_numeric(table.index.str.strip())
    table.columns = table.columns.str.strip()
    return table

[SAT](https://github.com/CMB-S4/DecadalSurveyReport/blob/83d362e2301b80444ed09c29293998cf5917d899/ScienceMeasurement/sciencemeasurement.tex#L336-L344), Table `2-1`

In [20]:
table21 = parse_table("""Frequency & 20 & 30 & 40 & 85 & 95 & 145 & 155 & 220 & 270
Angular resolution (arcmin) & 11.0 & 72.8 & 72.8 & 25.5 &  22.7 & 25.5 & 22.7 & 13.0 & 13.0
Total survey weight / $10^6$ ($\mu$K$^{-2}$) & 0.12& 0.69 & 0.43 & 11.0 & 14.1& 5.7 & 4.8 & 0.71 & 0.24
$Q/U$ rms ($\mu$K-arcmin)  & 8.4 & 3.5 & 4.5 & 0.88 & 0.78 & 1.2 & 1.3 & 3.5 & 6.0""")

[Delensing survey LAT Pole](https://github.com/CMB-S4/DecadalSurveyReport/blob/master/ScienceMeasurement/sciencemeasurement.tex#L445-L448)

In [21]:
table22 = parse_table("""Frequency (GHz) & 20 & 27 & 39 & 93 & 145 & 225 & 278
Angular resolution (arcmin) & 11.0 & 7.3 & 5.5 & 2.3 & 1.5 & 1.0 & 0.8
White noise level $E$/$B$ ($\mu$K-arcmin) & 8.4 & 5.0 & 4.5 & 0.68 & 0.96 & 5.7 & 9.8""")

[LAT Chile](https://github.com/CMB-S4/DecadalSurveyReport/blob/master/ScienceMeasurement/sciencemeasurement.tex#L523-L530)

In [22]:
table23 = parse_table("""Frequency (GHz) & 27 & 39 & 93 & 145 & 225 & 278
Angular resolution (arcmin) & 7.4 & 5.1 & 2.2 & 1.4 & 1.0 & 0.9 
Total survey weight ($TT$)/$10^6$ [$\mu$K$^{-2}$]  & 0.22 & 0.68 & 26.3 & 26.3 & 2.2 & 0.38 
White noise level for $TT$ ($\mu$K-arcmin) &21.8 &  12.4&   2.0&    2.0&    6.9&    16.7
White noise level $E$/$B$ ($\mu$K-arcmin) & 30.8& 17.6&   2.9&    2.8&    9.8&    23.6""")

In [23]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [24]:
for d in s4:
    freq = int(np.round(d["center_frequency"].value))
    if d["telescope"] == "SAT":
        nearest_freq = find_nearest(table21.index, freq)
        if nearest_freq != freq:
            print("Using {} instead of {} for {}".format(nearest_freq, freq, d["band"]))
            freq = nearest_freq
        d["pole_pol_sensitivity_DSR"] = table21.loc[freq]["$Q/U$ rms ($\mu$K-arcmin)"] * u.uK*u.arcmin
    else:
        nearest_freq = find_nearest(table22.index, freq)
        if nearest_freq != freq:
            print("Using {} for {} for {}".format(nearest_freq, d["center_frequency"].value, d["band"]))
            freq = nearest_freq
        if "ULF" not in d["band"]: # no 20GHz in Chile
            d["chile_temp_sensitivity_DSR"] = table23.loc[freq]["White noise level for $TT$ ($\mu$K-arcmin)"]* u.uK*u.arcmin
            d["chile_pol_sensitivity_DSR"] = table23.loc[freq]["White noise level $E$/$B$ ($\mu$K-arcmin)"]* u.uK*u.arcmin
        d["pole_pol_sensitivity_DSR"] = table22.loc[freq]["White noise level $E$/$B$ ($\mu$K-arcmin)"]* u.uK*u.arcmin

Using 27 for 25.75 for LFL1
Using 27 for 25.75 for LFPL1
Using 30 instead of 26 for LFS1
Using 40 instead of 39 for LFS2
Using 93 for 91.5 for MFL1
Using 145 for 148.5 for MFL2
Using 93 for 91.5 for MFPL1
Using 145 for 148.5 for MFPL2
Using 225 for 227.0 for HFL1
Using 278 for 285.5 for HFL2
Using 225 for 227.0 for HFPL1
Using 278 for 285.5 for HFPL2
Using 220 instead of 227 for HFS1
Using 270 instead of 286 for HFS2


# Create the astropy Table

In [25]:
import astropy.units as u

In [26]:
from astropy.table import QTable, Table

In [27]:
s4[0]

OrderedDict([('band', 'ULFPL1'),
             ('telescope', 'LAT'),
             ('center_frequency', <Quantity 20. GHz>),
             ('fwhm', <Quantity 10. arcmin>),
             ('detectors_per_tube', 54),
             ('NET', <Quantity 329.2 s(1/2) uK>),
             ('pole_pol_sensitivity_DSR', <Quantity 8.4 arcmin uK>)])

In [28]:
t = QTable(rows=s4, masked=True, names=list(s4[1]))
t.add_index("band")

In [29]:
t

band,telescope,center_frequency,fwhm,detectors_per_tube,NET,chile_temp_sensitivity_DSR,chile_pol_sensitivity_DSR,pole_pol_sensitivity_DSR
,,GHz,arcmin,,s(1/2) uK,arcmin uK,arcmin uK,arcmin uK
str6,str3,float64,float64,int64,float64,float64,float64,float64
ULFPL1,LAT,20.0,10.0,54,329.2,0.0,0.0,8.4
LFL1,LAT,25.75,7.4,96,287.4,21.8,30.8,5.0
LFL2,LAT,38.75,5.1,96,241.4,12.4,17.6,4.5
LFPL1,LAT,25.75,7.4,96,278.6,21.8,30.8,5.0
LFPL2,LAT,38.75,5.1,96,268.6,12.4,17.6,4.5
LFS1,SAT,25.75,72.8,288,169.2,0.0,0.0,3.5
LFS2,SAT,38.75,72.8,288,204.3,0.0,0.0,4.5
MFL1,LAT,91.5,2.2,864,274.5,2.0,2.9,0.68


In [30]:
t.loc["MFLS1"]["NET"]

<Quantity 290. s(1/2) uK>

In [31]:
t.loc["MFLS1"]["NET"]

<Quantity 290. s(1/2) uK>

In [32]:
t.meta["comments"] = ipac_warning

In [33]:
t.write("../instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac", overwrite=True)

In [34]:
c = QTable.read("../instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac" )

In [35]:
c.add_index("band")

In [36]:
c.loc["LFS1"]

band,telescope,center_frequency,fwhm,detectors_per_tube,NET,chile_temp_sensitivity_DSR,chile_pol_sensitivity_DSR,pole_pol_sensitivity_DSR
,,GHz,arcmin,,s(1/2) uK,arcmin uK,arcmin uK,arcmin uK
str6,str3,float64,float64,int64,float64,float64,float64,float64
LFS1,SAT,25.75,72.8,288,169.2,0.0,0.0,3.5


In [37]:
cat ../instrument_model/cmbs4_instrument_model.tbl

\ Text file in IPAC table format, read with astropy
\ from astropy.table import QTable
\ QTable.read('filename.tbl', format='ascii.ipac')
|  band|telescope|center_frequency|  fwhm|detectors_per_tube|      NET|chile_temp_sensitivity_DSR|chile_pol_sensitivity_DSR|pole_pol_sensitivity_DSR|
|  char|     char|          double|double|              long|   double|                    double|                   double|                  double|
|      |         |             GHz|arcmin|                  |s(1/2) uK|                 arcmin uK|                arcmin uK|               arcmin uK|
|  null|     null|            null|  null|              null|     null|                      null|                     null|                    null|
 ULFPL1       LAT             20.0   10.0                 54     329.2                        0.0                       0.0                      8.4 
   LFL1       LAT            25.75    7.4                 96     287.4                       21.8               